#Librerías

In [ ]:
import streamlit as st
import pandas as pd
from scipy.io import arff
from ludwig.api import LudwigModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import os
import tempfile
import json
import matplotlib.pyplot as plt
import numpy as np
from io import StringIO
import yaml

#Diseño y estilos CSS personalizados

In [ ]:
st.set_page_config(page_title="Entrenador Ludwig", layout="centered")

# Estilos CSS
st.markdown("""
    <style>
    .stApp {
        background-color: #f0f4f8;
        font-family: 'Segoe UI', sans-serif;
        padding: 2rem;
    }

    .stButton>button, .stDownloadButton>button {
        background-color: #ffd6d6;
        color: #d32f2f;
        border-radius: 8px;
        transition: 0.3s ease;
        border: none;
    }

    .stButton>button:hover, .stDownloadButton>button:hover {
        background-color: #ffbdbd;
        color: #b71c1c;
        transform: scale(1.02);
    }

    .stButton>button:focus, .stDownloadButton>button:focus {
        outline: none !important;
        box-shadow: none !important;
    }

    .stButton>button:active,
    .stDownloadButton>button:active,
    .stButton>button:active *,
    .stDownloadButton>button:active * {
        color: #b71c1c !important;
        background-color: #ffaaaa !important;
        outline: none !important;
        box-shadow: none !important;
        border: none !important;
        text-shadow: none !important;
        filter: none !important;
        -webkit-text-fill-color: #b71c1c !important;
    }

    h1, h2, h3 {
        color: #2c3e50;
    }
    </style>
""", unsafe_allow_html=True)



#Inicio y variables globales

In [ ]:
st.title("\U0001F393 Entrenador de modelos con Ludwig")
st.markdown("Carga un dataset `.arff` o archivos `.train`/`.test` de Excel junto con un `.yml`. Si el `.test` no tiene la columna objetivo, también se pedirá el `sample_submission.csv` antes de entrenar.")

df = None
train_df = None
test_df = None
sample_df = None
target_columns = []
output_type = ""
has_target = False
temp_yaml_path = None
yml_config = None

#Carga de archivos

In [ ]:
# Cargar tipo de archivo
file_type = st.radio("Tipo de archivo de datos:", [".arff", ".train/.test (Excel)"])
yaml_file = st.file_uploader("\u2699\ufe0f Sube tu archivo `.yml` de configuración", type=["yml", "yaml"])

# Función para cargar ARFF
def load_arff(arff_file):
    arff_content = arff_file.read().decode("utf-8")
    buffer = StringIO(arff_content)
    data, _ = arff.loadarff(buffer)
    df = pd.DataFrame(data)
    for col in df.select_dtypes(["object"]):
        df[col] = df[col].str.decode("utf-8")
    return df

# Leer YAML
if yaml_file is not None:
    yaml_bytes = yaml_file.read()
    with tempfile.NamedTemporaryFile(delete=False, suffix=".yml") as temp_yaml:
        temp_yaml.write(yaml_bytes)
        temp_yaml_path = temp_yaml.name
    yml_config = yaml.safe_load(yaml_bytes)
    target_columns = [out["name"] for out in yml_config.get("output_features", [])]
    output_type = yml_config["output_features"][0].get("type", "")

# Cargar archivos
if file_type == ".arff":
    arff_file = st.file_uploader("\U0001F4C2 Sube tu archivo `.arff`", type=["arff"])
    if arff_file and yaml_file:
        df = load_arff(arff_file)
        has_target = True
        st.success("Archivo .arff y configuración cargados correctamente.")

elif file_type == ".train/.test (Excel)":
    train_excel = st.file_uploader("\U0001F4C2 Sube el archivo de entrenamiento (.train)", type=["csv", "xlsx"])
    test_excel = st.file_uploader("\U0001F4C2 Sube el archivo de prueba (.test)", type=["csv", "xlsx"])
    if train_excel and test_excel and yaml_file:
        train_df = pd.read_csv(train_excel) if train_excel.name.endswith(".csv") else pd.read_excel(train_excel)
        test_df = pd.read_csv(test_excel) if test_excel.name.endswith(".csv") else pd.read_excel(test_excel)
        has_target = all(col in test_df.columns for col in target_columns)
        if not has_target:
            sample_submission = st.file_uploader("\U0001F4C4 El archivo `.test` no tiene columna objetivo. Sube `sample_submission.csv`", type=["csv"])
            if sample_submission:
                sample_df = pd.read_csv(sample_submission)
        st.success("Archivos cargados correctamente.")

#Entrenamiento y muestra de las métricas

In [ ]:
# Entrenamiento
if st.button("\U0001F680 Entrenar modelo"):
    if yaml_file is None or yml_config is None:
        st.error("Debes subir un archivo YAML de configuración.")
    else:
        with st.spinner("Entrenando el modelo..."):
            try:
                if df is not None:
                    train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
                    modo = ".arff (particionado 80/20)"
                elif train_df is not None and test_df is not None:
                    train_data, test_data = train_df, test_df
                    modo = ".train/.test (ya particionado)"
                else:
                    st.error("Faltan datos para entrenar el modelo.")
                    st.stop()

                st.info(f"Entrenando en modo: {modo}")

                model = LudwigModel(config=temp_yaml_path)
                model.train(dataset=train_data)

                predictions, _ = model.predict(dataset=test_data)

                if has_target:
                    st.success(f"\u2714\ufe0f El conjunto de test incluye la columna objetivo: {target_columns}")
                    eval_stats, _, _ = model.evaluate(dataset=test_data)
                    st.subheader("\U0001F4CA Métricas de Evaluación")
                    st.json(eval_stats)

                    if output_type == "category":
                        st.subheader("\U0001F9E9 Matriz de Confusión")
                        pred_col = [col for col in predictions.columns if col.endswith("_predictions")][0]
                        true_labels = test_data[target_columns[0]].values
                        pred_labels = predictions[pred_col].values

                        cm = confusion_matrix(true_labels, pred_labels, labels=np.unique(true_labels))
                        fig_cm = plt.figure(figsize=(6, 5))
                        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(true_labels))
                        disp.plot(cmap="Blues", ax=plt.gca())
                        st.pyplot(fig_cm)

                        st.subheader("\U0001F4CB Reporte detallado por clase")
                        report = classification_report(true_labels, pred_labels, output_dict=True)
                        st.json(report)

                    metrics_json = json.dumps(eval_stats, indent=2)
                    st.download_button("\u2B07\ufe0f Descargar métricas", metrics_json, "metricas.json", "application/json")

                else:
                    st.warning("\u26A0\ufe0f El conjunto de test no tiene columna objetivo. Se generará archivo para Kaggle.")
                    if sample_df is None:
                        st.error("\u274C Debes subir sample_submission.csv para este caso.")
                    else:
                        pred_column = [col for col in predictions.columns if col.endswith("_predictions")][0]
                        target_col = [col for col in sample_df.columns if col != "Id"][0]

                        if "Id" in sample_df.columns and sample_df.shape[0] == predictions.shape[0]:
                            submission = sample_df.copy()
                            if output_type == "number":
                                submission[target_col] = np.clip(predictions[pred_column].values, 0, None)
                            else:
                                submission[target_col] = predictions[pred_column].values
                            st.success("\u2714\ufe0f Archivo de envío generado correctamente.")
                            final_csv = submission.to_csv(index=False).encode("utf-8")
                            st.download_button("\u2B07\ufe0f Descargar submission.csv", final_csv, "submission.csv", "text/csv")
                        else:
                            st.error("\u274C El número de filas en test y sample_submission.csv no coincide.")

                st.subheader("\U0001F52E Predicciones")
                st.dataframe(predictions.head())
                pred_csv = predictions.to_csv(index=False).encode("utf-8")
                st.download_button("\u2B07\ufe0f Descargar predicciones", pred_csv, "predicciones.csv", "text/csv")

            except Exception as e:
                st.error(f"\u274C Error durante el entrenamiento: {e}")
            finally:
                if temp_yaml_path:
                    os.remove(temp_yaml_path)